In [1]:
import gym
import numpy as np
import random
import statistics as st

env=gym.make('MountainCar-v0')
env.reset()
states=np.round((env.observation_space.high - env.observation_space.low)*np.array([10,100])).astype(int)+1
actions=env.action_space.n
position=states[0]
velocity=states[1]
Q=np.random.uniform(low=-1,high=1,size=(position,velocity,actions))
eps=1
eps_min=0.01
eps_dec=0.0001
eps_max=1
discount=0.9
reward_hist=[]

iter_per_episode=200
max_episode=12000
Success_State=[]
Success_Action=[]

In [2]:
for episode in range(max_episode):
    states=[]
    actions=[]
    state=env.reset()
    states.append(state)
    state=np.round((state-env.observation_space.low)*np.array([10,100])).astype(int)
    
    position = state[0]
    velocity = state[1]
    total_reward=0
    for _ in range(iter_per_episode):
        eps_thresh=random.uniform(0,1)
        if eps_thresh < eps:
            action = np.random.randint(0,3)
        else:
            action = np.argmax(Q[position,velocity])
            
        actions.append(action)

        new_state,reward,done,info=env.step(action)
        states.append(new_state)
        total_reward+=reward
        new_adj = np.round((new_state-env.observation_space.low)*np.array([10,100])).astype(int)
        new_position = new_adj[0]
        new_velocity = new_adj[1]


        if new_state[0] >= 0.5:
            Q[position,velocity,action]=10
            Success_State.append(states)
            Success_Action.append(actions)
            

        else:
            delta = 0.2*(reward + discount*(np.max(Q[new_position,new_velocity]))-Q[position,velocity,action])
            Q[position,velocity,action]+=delta

        position = new_position
        velocity = new_velocity
        
        if done==True:
            break
    eps = (eps_max-eps_min)*np.exp(-eps_dec*episode)
    reward_hist.append(total_reward)

    if (episode%1000==0):
        print('Episode ',str(episode),'Average ',str(st.mean(reward_hist)))
        reward_hist=[]

    

Episode  0 Average  -200.0
Episode  1000 Average  -200.0
Episode  2000 Average  -200.0
Episode  3000 Average  -200.0
Episode  4000 Average  -200.0
Episode  5000 Average  -200.0
Episode  6000 Average  -200.0
Episode  7000 Average  -200.0
Episode  8000 Average  -199.995
Episode  9000 Average  -200.0
Episode  10000 Average  -199.902
Episode  11000 Average  -199.799


In [3]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(2,input_shape=(2,)))
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(3,activation=tf.nn.softmax))

model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])



In [4]:
for i in range(len(Success_Action)):
    x = np.array(Success_State[i][:-1])
    y = np.array(Success_Action[i])
    
    
    model.fit(x,y,epochs=3,verbose=1)

Epoch 1/3
195/195 [==============================] - 2s 10ms/step - loss: 1.0965 - acc: 0.3641
Epoch 2/3
195/195 [==============================] - 0s 267us/step - loss: 1.0814 - acc: 0.4308
Epoch 3/3
195/195 [==============================] - 0s 225us/step - loss: 1.0710 - acc: 0.4513
Epoch 1/3
196/196 [==============================] - 0s 270us/step - loss: 1.0940 - acc: 0.3827
Epoch 2/3
196/196 [==============================] - 0s 230us/step - loss: 1.0963 - acc: 0.3776
Epoch 3/3
196/196 [==============================] - 0s 270us/step - loss: 1.0940 - acc: 0.3929
Epoch 1/3
168/168 [==============================] - 0s 423us/step - loss: 1.0890 - acc: 0.3869
Epoch 2/3
168/168 [==============================] - 0s 304us/step - loss: 1.0858 - acc: 0.3810
Epoch 3/3
168/168 [==============================] - 0s 304us/step - loss: 1.0809 - acc: 0.4286
Epoch 1/3
174/174 [==============================] - 0s 287us/step - loss: 1.0825 - acc: 0.3506
Epoch 2/3
174/174 [======================

197/197 [==============================] - 0s 289us/step - loss: 0.9855 - acc: 0.5990
Epoch 3/3
197/197 [==============================] - 0s 269us/step - loss: 0.9827 - acc: 0.5888
Epoch 1/3
175/175 [==============================] - 0s 406us/step - loss: 1.0016 - acc: 0.5714
Epoch 2/3
175/175 [==============================] - 0s 280us/step - loss: 0.9972 - acc: 0.5771
Epoch 3/3
175/175 [==============================] - 0s 211us/step - loss: 0.9941 - acc: 0.5886
Epoch 1/3
174/174 [==============================] - 0s 281us/step - loss: 0.9506 - acc: 0.6322
Epoch 2/3
174/174 [==============================] - 0s 259us/step - loss: 0.9348 - acc: 0.6207
Epoch 3/3
174/174 [==============================] - 0s 264us/step - loss: 0.9162 - acc: 0.6437


In [5]:
for _ in range(10):
    state=env.reset()
    for i in range (200):
                
        action = np.argmax(model.predict(np.array(state).reshape(1,2)))
        new_state,rew,done,info = env.step(action)
        env.render()
        state=new_state
        if done==True:

            break
env.close()


In [ ]:
model.save('C:\\Users\\Rachit\\Desktop\\Gym\\MountainCar.h5')